In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def analisar_previsoes_travadas(previsoes_path):
    """Analisa se as previsões estão 'travadas' (baixa variação)"""
    if not os.path.exists(previsoes_path):
        print(f"Arquivo não encontrado: {previsoes_path}")
        return None
    
    df = pd.read_csv(previsoes_path)
    if 'Data' in df.columns:
        df['Data'] = pd.to_datetime(df['Data'])
    
    # Extrair preços reais e previstos
    preco_real = df['Preço Real'].values
    preco_previsto = df['Preço Previsto'].values
    
    # Calcular variação percentual
    var_real = np.std(preco_real) / np.mean(preco_real) if np.mean(preco_real) != 0 else 0
    var_previsto = np.std(preco_previsto) / np.mean(preco_previsto) if np.mean(preco_previsto) != 0 else 0
    
    # Calcular proporção de variação (previsto/real)
    prop_var = var_previsto / var_real if var_real != 0 else 0
    
    # Calcular métricas
    mae = mean_absolute_error(preco_real, preco_previsto)
    rmse = np.sqrt(mean_squared_error(preco_real, preco_previsto))
    r2 = r2_score(preco_real, preco_previsto)
    
    # Detectar se as previsões estão travadas
    travado = prop_var < 0.3  # Se a variação da previsão for menos de 30% da variação real
    
    resultados = {
        'variacao_real': var_real,
        'variacao_previsto': var_previsto,
        'proporcao_variacao': prop_var,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'travado': travado
    }
    
    return resultados

def plotar_importancia_macromicro(ticker, pasta_dados, resultados_dir):
    """Analisa a importância de features macro e micro econômicas"""
    # Carregar dados originais
    arquivo = f"{pasta_dados}/{ticker}_dados_macro_micro.csv"
    
    if not os.path.exists(arquivo):
        print(f"Dados não encontrados para {ticker}")
        return
    
    # Carregar dados
    df = pd.read_csv(arquivo, index_col=0)
    df.index = pd.to_datetime(df.index)
    
    # Separar features macro e micro
    features_macro = ['TaxaCambio', 'Selic', 'PIB', 'IPCA']
    features_micro = ['ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']
    
    # Verificar quais features existem no dataframe
    features_macro_existentes = [f for f in features_macro if f in df.columns]
    features_micro_existentes = [f for f in features_micro if f in df.columns]
    
    if not features_macro_existentes and not features_micro_existentes:
        print(f"Nenhuma feature macro/micro encontrada para {ticker}")
        return
    
    # Calcular correlações com o preço alvo
    correlacoes = {}
    target = 'Close_Target' if 'Close_Target' in df.columns else 'Preço' if 'Preço' in df.columns else None
    
    if target is None:
        print(f"Coluna de preço alvo não encontrada para {ticker}")
        return
    
    # Calcular correlações
    for feature in features_macro_existentes + features_micro_existentes:
        if feature in df.columns:
            correlacoes[feature] = df[feature].corr(df[target])
    
    # Criar gráficos
    plt.figure(figsize=(12, 8))
    
    # Gráfico de correlações
    bars = plt.bar(correlacoes.keys(), correlacoes.values())
    
    # Colorir barras por tipo (macro/micro)
    for i, feature in enumerate(correlacoes.keys()):
        if feature in features_macro_existentes:
            bars[i].set_color('blue')
        else:
            bars[i].set_color('green')
    
    plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
    plt.title(f'Correlação de Features Macro/Micro com Preço - {ticker}')
    plt.ylabel('Correlação de Pearson')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Criar legenda personalizada
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', label='Features Macro'),
        Patch(facecolor='green', label='Features Micro')
    ]
    plt.legend(handles=legend_elements)
    
    # Salvar gráfico
    plt.savefig(f"{resultados_dir}/{ticker}_correlacoes_macromicro.png")
    plt.close()
    
    return correlacoes

def analisar_todos_tickers(pasta_dados, resultados_dir):
    """Analisa todos os tickers e suas previsões"""
    os.makedirs(resultados_dir, exist_ok=True)
    
    # Obter lista de tickers dos arquivos disponíveis
    arquivos = glob.glob(f"{pasta_dados}/*_dados_macro_micro.csv")
    tickers = [os.path.basename(arquivo).split('_')[0] for arquivo in arquivos]
    
    # Analisar cada ticker
    resultados_resumidos = []
    for ticker in tickers:
        print(f"\nAnalisando {ticker}...")
        
        # Plotar importância das features macro/micro
        correlacoes = plotar_importancia_macromicro(ticker, pasta_dados, resultados_dir)
        
        # Verificar previsões travadas (buscar arquivos de previsão)
        arquivos_previsao = glob.glob(f"resultados_lstm_cv_macro_FUNDAMENTUS_tratados/{ticker}_Janela_*_previsoes_teste_final.csv")
        
        for arquivo in arquivos_previsao:
            janela = arquivo.split('_')[-3]  # Extrair número da janela
            resultados = analisar_previsoes_travadas(arquivo)
            
            if resultados:
                resultado_resumido = {
                    'Ticker': ticker,
                    'Janela': janela,
                    'R2': resultados['r2'],
                    'RMSE': resultados['rmse'],
                    'Variacao_Real': resultados['variacao_real'],
                    'Variacao_Previsto': resultados['variacao_previsto'],
                    'Proporcao_Variacao': resultados['proporcao_variacao'],
                    'Travado': 'Sim' if resultados['travado'] else 'Não'
                }
                resultados_resumidos.append(resultado_resumido)
                
                print(f"  Janela {janela}: R² = {resultados['r2']:.4f}, RMSE = {resultados['rmse']:.4f}")
                print(f"  Variação Real: {resultados['variacao_real']:.4f}, Variação Previsto: {resultados['variacao_previsto']:.4f}")
                print(f"  Previsões travadas? {'Sim' if resultados['travado'] else 'Não'}")
    
    # Salvar resumo
    if resultados_resumidos:
        df_resumo = pd.DataFrame(resultados_resumidos)
        df_resumo.to_csv(f"{resultados_dir}/resumo_previsoes_macromicro.csv", index=False)
        print(f"\nResumo salvo em {resultados_dir}/resumo_previsoes_macromicro.csv")

def plotar_comparacao_macromicro():
    """Plota comparação entre modelos usando apenas features macro vs micro vs ambas"""
    # Implementação futura - necessitaria de experimentos adicionais
    pass

if __name__ == "__main__":
    pasta_dados = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dados_unificados"
    resultados_dir = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\analise_macromicro"
    
    analisar_todos_tickers(pasta_dados, resultados_dir)



Analisando BEEF3.SA...

Analisando BRFS3.SA...

Analisando CSNA3.SA...

Analisando GGBR4.SA...

Analisando JBSS3.SA...

Analisando PETR4.SA...

Analisando SOJA3.SA...

Analisando SUZB3.SA...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



Analisando VALE3.SA...
